In [ ]:
%%capture
!pip install timm huggingface_hub["fastai"]

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

# Colab-specific code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install timm==0.9.16

## Clone repository to cloned_repository folder and cd there.

In [ ]:
!git clone https://github.com/mstaczek/cnn-vs-transformers-msc-thesis.git cloned_repository
import os
os.chdir("/content/cloned_repository/codes")
os.getcwd()

Cloning into 'cloned_repository'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (550/550), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 550 (delta 304), reused 423 (delta 177), pack-reused 0
Receiving objects: 100% (550/550), 7.74 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (304/304), done.


'/content/cloned_repository/codes'

## Download data

In [ ]:
import timm
import torch
import os
from datetime import datetime
from fastai.vision.all import *
from huggingface_hub import push_to_hub_fastai


path = untar_data(URLs.IMAGENETTE_320)
# path = "..\datasets\imagenette2"

### Helper functions

In [ ]:
def train_on_imagenette(model_name, models_output, epochs, batch_size):
    from fastai.vision.all import accuracy
    os.makedirs(models_output, exist_ok=True)
    dls = ImageDataLoaders.from_folder(path, valid='val', bs=batch_size, # batch size
                                    item_tfms=RandomResizedCrop((224,224), min_scale=0.35),
                                    batch_tfms=Normalize.from_stats(*imagenet_stats))
    # creates a pretrained model with a new classification head for same number of classes as there is in dataloader
    learn = vision_learner(dls, model_name, metrics=accuracy)
    learn.fine_tune(epochs)
    models_accuracy = learn.validate()[1]
    filename = f"{models_output}/{models_accuracy:.4f}.pth"
    torch.save(learn.model, filename)
    print("Saved best model to ", filename)
    return models_accuracy

def copy_to_drive(model_name, models_output, models_accuracy):
    source_folder = f"{models_output}/*"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    drive_results_folder = f"/content/drive/MyDrive/results/imagenette_{timestamp}_{model_name}_{models_accuracy:.4f}"
    os.makedirs(drive_results_folder, exist_ok=True)
    os.system(f"cp -r {source_folder} {drive_results_folder}")

def upload_to_huggingface(learner, model_name):
    push_to_hub_fastai(learner=learn, repo_id=f"bnemu/imagenette2-{model_name}")

## Model training

In [ ]:
model_name = 'convnextv2_nano.fcmae_ft_in22k_in1k_384' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size) # 250s
copy_to_drive(model_name, models_output, models_accuracy)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,time
0,0.171350,0.028095,0.992102,01:30


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch,train_loss,valid_loss,accuracy,time
0,0.044376,0.017992,0.994395,02:14


Saved best model to  ../trained_models/imagenette2/convnextv2_nano.fcmae_ft_in22k_in1k_384/0.9944.pth


In [ ]:
model_name = 'deit_small_patch16_224.fb_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size) # 230s
copy_to_drive(model_name, models_output, models_accuracy)

epoch,train_loss,valid_loss,accuracy,time
0,0.222902,0.014987,0.995924,01:33


epoch,train_loss,valid_loss,accuracy,time
0,0.045585,0.012030,0.996433,01:52


Saved best model to  ../trained_models/imagenette2/deit_small_patch16_224.fb_in1k/0.9964.pth


In [ ]:
model_name = 'deit_tiny_patch16_224.fb_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size) # 140s
copy_to_drive(model_name, models_output, models_accuracy)

epoch,train_loss,valid_loss,accuracy,time
0,0.412480,0.051848,0.984968,01:00


epoch,train_loss,valid_loss,accuracy,time
0,0.121927,0.042227,0.987516,01:04


Saved best model to  ../trained_models/imagenette2/deit_tiny_patch16_224.fb_in1k/0.9875.pth


In [ ]:
model_name = 'densenet121.tv_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size) # 250s
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.203987,0.059621,0.980637,01:36


epoch,train_loss,valid_loss,accuracy,time
0,0.099075,0.037054,0.988280,02:02


Saved best model to  ../trained_models/imagenette2/densenet121.tv_in1k/0.9883.pth


In [ ]:
model_name = 'efficientnet_b3.ra2_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size) # 230s
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.206404,0.071619,0.978599,01:33


epoch,train_loss,valid_loss,accuracy,time
0,0.109592,0.058234,0.981656,01:55


Saved best model to  ../trained_models/imagenette2/efficientnet_b3.ra2_in1k/0.9817.pth


In [ ]:
model_name = 'efficientnet_b4.ra2_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 32

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,time
0,0.262666,0.091346,0.968917,01:58


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


epoch,train_loss,valid_loss,accuracy,time
0,0.146190,0.060124,0.979873,02:48


Saved best model to  ../trained_models/imagenette2/efficientnet_b4.ra2_in1k/0.9799.pth


In [ ]:
model_name = 'mobilenetv3_large_100.ra_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: running_mean should contain 1920 elements not 2560

In [ ]:
model_name = 'pvt_v2_b2.in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.143815,0.012456,0.997197,01:49


epoch,train_loss,valid_loss,accuracy,time
0,0.027875,0.010383,0.997707,05:11


Saved best model to  ../trained_models/imagenette2/pvt_v2_b2.in1k/0.9977.pth


In [ ]:
model_name = 'res2net50_14w_8s.in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size) # 290s
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.148331,0.031113,0.990318,01:51


epoch,train_loss,valid_loss,accuracy,time
0,0.078922,0.016608,0.994650,02:23


Saved best model to  ../trained_models/imagenette2/res2net50_14w_8s.in1k/0.9946.pth


In [ ]:
model_name = 'resnet18d.ra2_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.335243,0.074684,0.978344,00:57


epoch,train_loss,valid_loss,accuracy,time
0,0.163166,0.062818,0.979873,01:00


Saved best model to  ../trained_models/imagenette2/resnet18d.ra2_in1k/0.9799.pth


In [ ]:
model_name = 'resnet50d.ra2_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/103M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.161016,0.041686,0.987516,01:31


epoch,train_loss,valid_loss,accuracy,time
0,0.079490,0.028668,0.991592,01:57


Saved best model to  ../trained_models/imagenette2/resnet50d.ra2_in1k/0.9916.pth


In [ ]:
model_name = 'resnext50_32x4d.a1h_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/100M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.179947,0.123100,0.982675,01:51


epoch,train_loss,valid_loss,accuracy,time
0,0.087987,0.051945,0.988280,02:26


Saved best model to  ../trained_models/imagenette2/resnext50_32x4d.a1h_in1k/0.9883.pth


In [ ]:
model_name = 'swin_tiny_patch4_window7_224.ms_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 32

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: running_mean should contain 14 elements not 1536

In [ ]:
model_name = 'vit_base_patch32_224.augreg_in21k_ft_in1k' # model name from timm library
models_output = f'../trained_models/imagenette2/{model_name}'

epochs = 1
batch_size = 64

models_accuracy = train_on_imagenette(model_name, models_output, epochs, batch_size)
copy_to_drive(model_name, models_output, models_accuracy)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.278740,0.060582,0.979108,01:12


epoch,train_loss,valid_loss,accuracy,time
0,0.074498,0.034800,0.988790,01:37


Saved best model to  ../trained_models/imagenette2/vit_base_patch32_224.augreg_in21k_ft_in1k/0.9888.pth


---

In [ ]:
import torch
loaded_model = torch.load("../trained_models/imagenette/ResNet18/0.93_torch-save.pth")
loaded_model(torch.randn(1, 3, 224, 224).to('cuda')).shape